In [1]:
import sys
import numpy as np

# TensorFlow and tf.keras
import tensorflow.compat.v1 as tf
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras import utils as np_utils 
from keras.preprocessing.text import Tokenizer

In [2]:
# loading the text file and convert texts to lowercase
with open('/content/drive/MyDrive/data/shakespeare.txt', 'r') as f:
    text = f.read().lower()

# create a set of all unique characters in the text
chars = sorted(list(set(text)))

In [3]:
# create a dictionary to map characters to integers and vice versa
char_to_int = dict((c, i) for i, c in enumerate(chars))
idx_to_char = {i: c for i, c in enumerate(chars)}

In [4]:
# size of the vocabulary (number of unique characters)
vocab_size = len(chars)
vocab_size

38

In [5]:
#Number of characters
n_chars = len(text)
n_chars

93677

In [6]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 20
X = []
Y = []
for i in range(0, n_chars-seq_length):
 seq_in = text[i:i + seq_length]
 seq_out = text[i + seq_length]
 X.append([char_to_int[char] for char in seq_in])
 Y.append(char_to_int[seq_out])

In [7]:
n_patterns = len(X)
print ("Total Patterns: "), n_patterns

Total Patterns: 


(None, 93657)

In [8]:
X = np.reshape(X, (n_patterns, seq_length, 1))

X = X / float(vocab_size)   # normalize
y = to_categorical(Y)   # one hot encode the output variable

In [9]:
import tensorflow.keras as keras

# define the LSTM model

model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [14]:
# define the checkpoint
filepath="/content/drive/MyDrive/Weights/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [15]:
model.compile(loss='categorical_crossentropy', optimizer="adam")

In [16]:
model.fit(X, y, epochs=1000, batch_size=64, callbacks=callbacks_list)

Epoch 1/1000
1459/1464 [============================>.] - ETA: 0s - loss: 2.4192
Epoch 1: loss improved from inf to 2.41878, saving model to /content/drive/MyDrive/Weights/weights-improvement-01-2.4188.hdf5
1464/1464 [==============================] - 14s 7ms/step - loss: 2.4188
Epoch 2/1000
1462/1464 [============================>.] - ETA: 0s - loss: 2.3748
Epoch 2: loss improved from 2.41878 to 2.37453, saving model to /content/drive/MyDrive/Weights/weights-improvement-02-2.3745.hdf5
1464/1464 [==============================] - 10s 7ms/step - loss: 2.3745
Epoch 3/1000
1458/1464 [============================>.] - ETA: 0s - loss: 2.3254
Epoch 3: loss improved from 2.37453 to 2.32532, saving model to /content/drive/MyDrive/Weights/weights-improvement-03-2.3253.hdf5
1464/1464 [==============================] - 12s 8ms/step - loss: 2.3253
Epoch 4/1000
1459/1464 [============================>.] - ETA: 0s - loss: 2.2675
Epoch 4: loss improved from 2.32532 to 2.26729, saving model to /conten

In [17]:
# load the network weights

filename = "/content/drive/MyDrive/Weights/weights-improvement-914-0.0221.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer="adam")